# Book Summarization 

5 Levels Of Summarization:

1. Summarize a couple sentences - Basic Prompt
2. Summarize a couple paragraphs - Prompt Templates
3. Summarize a couple pages - Map Reduce
4. Summarize an entire book - Best Representation Vectors
5. Summarize an unknown amount of text - Agents

The BRV Steps:

Load your book into a single text file
Split your text into large-ish chunks
Embed your chunks to get vectors
Cluster the vectors to see which are similar to each other and likely talk about the same parts of the book
Pick embeddings that represent the cluster the most (method: closest to each cluster centroid)
Summarize the documents that these embeddings represent

In [1]:
import urllib
import warnings
from pathlib import Path as p

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
# from langchain.document_loaders import PyPDFLoader
from langchain.llms import VertexAI

In [2]:
!pip install google-cloud-aiplatform>=1.38.0

zsh:1: 1.38.0 not found


In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY', '')

# Using MapReduce 

In [3]:
from langchain import OpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [15]:
!pip install PyPDF2

  Obtaining dependency information for PyPDF2 from https://files.pythonhosted.org/packages/8e/5e/c86a5643653825d3c913719e788e41386bee415c2b87b4f955432f2de6b2/pypdf2-3.0.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.5 MB/s eta 0:00:00a 0:00:01


In [18]:
!pip install pdfreader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 17.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for bitarray>=1.1.0 from https://files.pythonhosted.org/packages/06/60/c1a419f8abd0c9d2641e3e570fc63ad3a87a63ef88a362900e3254f780bc/bitarray-2.9.2-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for pycryptodome>=3.9.9 from https://files.pythonhosted.org/packages/ff/96/b0d494defb3346378086848a8ece5ddfd138a66c4a05e038fca873b2518c/pycryptodome-3.20.0-cp35-abi3-macosx_10_9_universal2.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 7.0 MB/s eta 0:00:0000:0100:01m
  Created wheel for pdfreader: filename=pdfreader-0.1.12-py3-none-any.whl size=134539 sha256=c8e116885476515f1e2e85e43fd103d630917fb4b1e2ea18e74b29e966a34ee8
  Stored in directory: /Users/riyajoshi/Library/Caches/pip/wheels/ae/2a/d7/534c5ec

In [4]:
# Path to your PDF file
book_pdf = 'Downloads/Harry_Potter_Movies/Chapters.csv'

content = []

with open(book_pdf, 'rb') as file:  # Note the 'rb' mode for reading PDF as binary
    content.append(file.read())

print(content[:100])

[b'Chapter ID,Chapter Name,Movie ID,Movie Chapter\r\n1,Doorstep Delivery,1,1\r\n2,The Vanishing Glass,1,2\r\n3,Letters from No One,1,3\r\n4,Keeper of the Keys,1,4\r\n5,Diagon Alley,1,5\r\n6,Gringotts,1,6\r\n7,Ollivanders,1,7\r\n8,The Boy Who Lived,1,8\r\n9,Platform 9&\xbe,1,9\r\n10,Ron and Hermione,1,10\r\n11,Welcome to Hogwarts,1,11\r\n12,The Sorting Hat,1,12\r\n13,Nick and Other Residents,1,13\r\n14,Potions and Parcels,1,14\r\n15,New Seeker,1,15\r\n16,Three-Headed Sentinel,1,16\r\n17,Facts and Feathers,1,17\r\n18,Mountain Troll,1,18\r\n19,Quidditch,1,19\r\n20,Interference Overcome,1,20\r\n21,Christmas Gift,1,21\r\n22,Cloaked in darkness,1,22\r\n23,The Mirror of Erised,1,23\r\n24,Norbert,1,24\r\n25,The Forbidden Forest,1,25\r\n26,Up to Something,1,26\r\n27,Through the Trapdoor,1,27\r\n28,Wizard\'s Chess,1,28\r\n29,Sacrifice Play,1,29\r\n30,Man With Two Faces,1,30\r\n31,Magic Touch,1,31\r\n32,Mark of Love,1,32\r\n33,House Cup Winner,1,33\r\n34,Not Really Going Home,1,34\r\n35,In a Cage

In [5]:
from langchain.llms import OpenAI 
llm = OpenAI(api_key="")

# decoding bytes to string 
text_content = [item.decode('utf-8', errors='replace') if isinstance(item, bytes) else item for item in content]
# Join the list into a single string
text = "\n".join(text_content)

# check text is of type string 
print(text[:500])  

# display number of tokens 
num_tokens = llm.get_num_tokens(text)
print(f"Number of tokens: {num_tokens}")

/Users/riyajoshi/miniconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


Chapter ID,Chapter Name,Movie ID,Movie Chapter
1,Doorstep Delivery,1,1
2,The Vanishing Glass,1,2
3,Letters from No One,1,3
4,Keeper of the Keys,1,4
5,Diagon Alley,1,5
6,Gringotts,1,6
7,Ollivanders,1,7
8,The Boy Who Lived,1,8
9,Platform 9&�,1,9
10,Ron and Hermione,1,10
11,Welcome to Hogwarts,1,11
12,The Sorting Hat,1,12
13,Nick and Other Residents,1,13
14,Potions and Parcels,1,14
15,New Seeker,1,15
16,Three-Headed Sentinel,1,16
17,Facts and Feathers,1,17
18,Mountain Troll,1,18

Number of tokens: 2532


In [32]:
!pip install pdfplumber

  Obtaining dependency information for pdfplumber from https://files.pythonhosted.org/packages/ee/ec/315015b6c4667d889cc14348c93e31f9fd43a6f61162b85c39e4a21018d5/pdfplumber-0.11.0-py3-none-any.whl.metadata
  Obtaining dependency information for pdfminer.six==20231228 from https://files.pythonhosted.org/packages/eb/9c/e46fe7502b32d7db6af6e36a9105abb93301fa1ec475b5ddcba8b35ae23a/pdfminer.six-20231228-py3-none-any.whl.metadata
  Obtaining dependency information for pypdfium2>=4.18.0 from https://files.pythonhosted.org/packages/42/ea/ae566f67fe82884d365ec574d3105d15df7baecf96d14f25d175fdcf4e73/pypdfium2-4.29.0-py3-none-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 5.0 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 2.6 MB/s eta 0:00:0000:0100:01


In [22]:
template = """
Please write a one sentence summary of the following chapters:

{text}
"""

prompt = PromptTemplate(
    input_variables=["text"],
    template=template
)

In [10]:
text_content = [item.decode('utf-8', errors='replace') if isinstance(item, bytes) else item for item in content]
# Join the list into a single string
text = "\n".join(text_content)

# check text is of type string 
print(text[:500])  

# display number of tokens 
num_tokens = llm.get_num_tokens(text)
print(f"Number of tokens: {num_tokens}")

Chapter ID,Chapter Name,Movie ID,Movie Chapter
1,Doorstep Delivery,1,1
2,The Vanishing Glass,1,2
3,Letters from No One,1,3
4,Keeper of the Keys,1,4
5,Diagon Alley,1,5
6,Gringotts,1,6
7,Ollivanders,1,7
8,The Boy Who Lived,1,8
9,Platform 9&�,1,9
10,Ron and Hermione,1,10
11,Welcome to Hogwarts,1,11
12,The Sorting Hat,1,12
13,Nick and Other Residents,1,13
14,Potions and Parcels,1,14
15,New Seeker,1,15
16,Three-Headed Sentinel,1,16
17,Facts and Feathers,1,17
18,Mountain Troll,1,18

Number of tokens: 2532


In [12]:
print(text[:100])

Chapter ID,Chapter Name,Movie ID,Movie Chapter
1,Doorstep Delivery,1,1
2,The Vanishing Glass,1,2



In [14]:
def approximate_token_count(text):
    # Very simple approximation: split the text into 'words' based on spaces and some punctuation
    # This is a crude approximation and will not exactly match OpenAI's tokenization, which is based on BPE.
    tokens = text.split()
    return len(tokens)

# Use the function to get an approximate token count
text = "\n".join(text_content)
num_tokens = approximate_token_count(text)
print(f"Estimated number of tokens: {num_tokens}")


Estimated number of tokens: 634


In [17]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n"], chunk_size=10000, chunk_overlap=500)
docs = text_splitter.create_documents([text])
print(docs)

[Document(page_content='Chapter ID,Chapter Name,Movie ID,Movie Chapter\r\n1,Doorstep Delivery,1,1\r\n2,The Vanishing Glass,1,2\r\n3,Letters from No One,1,3\r\n4,Keeper of the Keys,1,4\r\n5,Diagon Alley,1,5\r\n6,Gringotts,1,6\r\n7,Ollivanders,1,7\r\n8,The Boy Who Lived,1,8\r\n9,Platform 9&�,1,9\r\n10,Ron and Hermione,1,10\r\n11,Welcome to Hogwarts,1,11\r\n12,The Sorting Hat,1,12\r\n13,Nick and Other Residents,1,13\r\n14,Potions and Parcels,1,14\r\n15,New Seeker,1,15\r\n16,Three-Headed Sentinel,1,16\r\n17,Facts and Feathers,1,17\r\n18,Mountain Troll,1,18\r\n19,Quidditch,1,19\r\n20,Interference Overcome,1,20\r\n21,Christmas Gift,1,21\r\n22,Cloaked in darkness,1,22\r\n23,The Mirror of Erised,1,23\r\n24,Norbert,1,24\r\n25,The Forbidden Forest,1,25\r\n26,Up to Something,1,26\r\n27,Through the Trapdoor,1,27\r\n28,Wizard\'s Chess,1,28\r\n29,Sacrifice Play,1,29\r\n30,Man With Two Faces,1,30\r\n31,Magic Touch,1,31\r\n32,Mark of Love,1,32\r\n33,House Cup Winner,1,33\r\n34,Not Really Going Home,1,

In [24]:
summary_chain = load_summarize_chain(llm=llm, chain_type='map_reduce',
#                                      verbose=True # Set verbose=True if you want to see the prompts being used
                                    )

In [ ]:
output = summary_chain.invoke(docs[:500])

In [18]:
map_prompt_template = """
                      Write a summary of this chunk of text that includes the main points and any important details.
                      {text}
                      """

map_prompt = PromptTemplate(template=map_prompt_template, input_variables=["text"])

combine_prompt_template = """
                      Write a concise summary of the following text delimited by triple backquotes.
                      Return your response in bullet points which covers the key points of the text.
                      ```{text}```
                      BULLET POINT SUMMARY:
                      """

combine_prompt = PromptTemplate(
    template=combine_prompt_template, input_variables=["text"]
)

In [57]:
# use simple sequentail or sequentail chain (ouput of 1 and input of 2 to generate output 2 etc)

In [ ]:
map_reduce_chain = load_summarize_chain(
    vertex_llm_text,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=combine_prompt,
    return_intermediate_steps=True,
)

In [ ]:
output = summary_chain.invoke(docs)
print(output)

In [ ]:
map_reduce_outputs = map_reduce_chain({"input_documents": pages})

# Sentence Transformers to Optimize Model 

In [7]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

# Load a pre-trained Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Synonym dictionary for tag normalization 
synonym_dict = {
    "key": ["value"]
}

In [ ]:
# Normalized Tags
print(precompute_synonym_embeddings(synonym_dict, model))

# using Vertex AI